In [1]:
import pyspark
myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [2]:
import pandas as pd
import os
import numpy as np
%matplotlib inline


In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.functions import lit,col
from pyspark.sql import functions as F

In [5]:
trainDf = spark.read.format('com.databricks.spark.csv').options(header='true',inferschema='true').load(os.path.join
('data','heart.csv'))

In [9]:
trainDf.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- cp: integer (nullable = true)
 |-- trestbps: integer (nullable = true)
 |-- chol: integer (nullable = true)
 |-- fbs: integer (nullable = true)
 |-- restecg: integer (nullable = true)
 |-- thalach: integer (nullable = true)
 |-- exang: integer (nullable = true)
 |-- oldpeak: double (nullable = true)
 |-- slope: integer (nullable = true)
 |-- ca: integer (nullable = true)
 |-- thal: integer (nullable = true)
 |-- target: integer (nullable = true)



In [10]:
trainDf.show()

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|
| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|
| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|
| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|
| 57|  1|  0|     140| 192|  0|      1|    148|    0|    0.4|    1|  0|   1|     1|
| 56|  0|  1|     140| 294|  0|      0|    153|    0|    1.3|    1|  0|   2|     1|
| 44|  1|  1|     120| 263|  0|      1|    173|    0|    0.0|    2|  0|   3|     1|
| 52|  1|  2|     172| 199|  1|      1|    162|    0|    0.5|    2|  0|   3|

# drop unnecessary columns

In [13]:
df = trainDf.withColumn("target1",trainDf['target'].cast("double")).drop('target')

In [15]:
df.show(10)

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+-------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target1|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+-------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|    1.0|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|    1.0|
| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|    1.0|
| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|    1.0|
| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|    1.0|
| 57|  1|  0|     140| 192|  0|      1|    148|    0|    0.4|    1|  0|   1|    1.0|
| 56|  0|  1|     140| 294|  0|      0|    153|    0|    1.3|    1|  0|   2|    1.0|
| 44|  1|  1|     120| 263|  0|      1|    173|    0|    0.0|    2|  0|   3|    1.0|
| 52|  1|  2|     172| 199|  1|      1|    162|    0|    0.5|    

In [16]:
trainDf.show(10)

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|
| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|
| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|
| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|
| 57|  1|  0|     140| 192|  0|      1|    148|    0|    0.4|    1|  0|   1|     1|
| 56|  0|  1|     140| 294|  0|      0|    153|    0|    1.3|    1|  0|   2|     1|
| 44|  1|  1|     120| 263|  0|      1|    173|    0|    0.0|    2|  0|   3|     1|
| 52|  1|  2|     172| 199|  1|      1|    162|    0|    0.5|    2|  0|   3|